In [ ]:
import io
import requests
import docx

In [6]:
# def clean_line(line):
#     line = line.strip()
#     line = line.strip('\uFEFF')#explain 
#     return line

# def read_faq(file_id):
#     url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
#     response = requests.get(url)
#     response.raise_for_status()
    
#     with io.BytesIO(response.content) as f_in:
#         doc = docx.Document(f_in)

#     questions = []

#     question_heading_style = 'heading 2'
#     section_heading_style = 'heading 1'
    
#     heading_id = ''
#     section_title = ''
#     question_title = ''
#     answer_text_so_far = ''
     
#     for p in doc.paragraphs:
#         style = p.style.name.lower()
#         p_text = clean_line(p.text)
    
#         if len(p_text) == 0:
#             continue
    
#         if style == section_heading_style:
#             section_title = p_text
#             continue
    
#         if style == question_heading_style:
#             answer_text_so_far = answer_text_so_far.strip()
#             if answer_text_so_far != '' and section_title != '' and question_title != '':
#                 questions.append({
#                     'text': answer_text_so_far,
#                     'section': section_title,
#                     'question': question_title,
#                 })
#                 answer_text_so_far = ''
    
#             question_title = p_text
#             continue
        
#         answer_text_so_far += '\n' + p_text
    
#     answer_text_so_far = answer_text_so_far.strip()
#     if answer_text_so_far != '' and section_title != '' and question_title != '':
#         questions.append({
#             'text': answer_text_so_far,
#             'section': section_title,
#             'question': question_title,
#         })

#     return questions
import json

def clean_line(line):
    """Removes unnecessary whitespace and BOM characters from a line."""
    line = line.strip()
    line = line.strip('\uFEFF')
    return line

def read_law_text(file_path):
    """Reads and parses a Vietnamese law document, structuring it into JSON format."""
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    law_data = {
        "law_title": "Luật Giáo Dục",
        "law_number": "43/2019/QH14",
        "chapters": []
    }

    current_chapter = None
    current_article = None

    for line in lines:
        line = clean_line(line)

        # Skip empty lines
        if not line:
            continue

        # Detect chapter
        if line.startswith("Chương"):
            if current_chapter:
                law_data["chapters"].append(current_chapter)
            current_chapter = {
                "chapter_title": line,
                "articles": []
            }

        # Detect article
        elif line.startswith("Điều"):
            if current_article:
                current_chapter["articles"].append(current_article)
            current_article = {
                "article_number": line.split(".")[0].replace("Điều ", "").strip(),
                "title": line.split(". ", 1)[1] if ". " in line else "",
                "content": ""
            }

        # Accumulate content for current article
        else:
            if current_article:
                current_article["content"] += line + " "

    # Append last article and chapter
    if current_article:
        current_chapter["articles"].append(current_article)
    if current_chapter:
        law_data["chapters"].append(current_chapter)

    return law_data

# Define the file path to your text file
file_path = "translated_luat_giao_duc.txt"
law_json = read_law_text(file_path)

# Save the output as a JSON file
with open("translated_luat_giao_duc.json", "w", encoding="utf-8") as f:
    json.dump(law_json, f, ensure_ascii=False, indent=4)

print("Conversion to JSON complete. Output saved as 'translated_luat_giao_duc.json'.")


Conversion to JSON complete. Output saved as 'translated_luat_giao_duc.json'.


In [ ]:
# faq_documents = {
#     'data-engineering-zoomcamp': '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw',
#     'machine-learning-zoomcamp': '1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8',
#     'mlops-zoomcamp': '12TlBfhIiKtyBv8RnsoJR6F72bkPDGEvPOItJIxaEzE0',
# }

In [ ]:
documents = []

for course, file_id in faq_documents.items():
    print(course)
    course_documents = read_faq(file_id)
    documents.append({'course': course, 'documents': course_documents})

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [2]:
import json

In [ ]:

# with open('documents.json', 'wt') as f_out:
#     json.dump(documents, f_out, indent=2)
with open('luat_giao_duc.json', 'wt') as f_out:
    json.dump(law_json, f_out, indent=2)


[
  {
    "article_number": "1",
    "title": "Phạm vi điều chỉnh",
    "content": "Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. "
  },
  {
    "article_number": "2",
    "title": "Mục tiêu giáo dục",
    "content": "Mục tiêu giáo dục nhằm phát triển toàn diện con người Việt Nam có đạo đức, tri thức, văn hóa, sức khỏe, thẩm mỹ và nghề nghiệp; có phẩm chất, năng lực và ý thức công dân; có lòng yêu nước, tinh thần dân tộc, trung thành với lý tưởng độc lập dân tộc và chủ nghĩa xã hội; phát huy tiềm năng, khả năng sáng tạo của mỗi cá nhân; nâng cao dân trí, phát triển nguồn nhân lực, bồi dưỡng nhân tài, đáp ứng yêu cầu của sự nghiệp xây dựng, bảo vệ Tổ quốc và hội nhập quốc tế. "


In [7]:
!head luat_giao_duc.json

"articles"